In [ ]:
import logging

# Configure logging to see info messages
logging.basicConfig(level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')

In [ ]:
from pyinstrument import Profiler


profiler = Profiler()

with profiler:
    pass

profiler.write_html('../temp/features.html')

In [ ]:
from labda_parsers import Sens
from acti_motus import Features

features = Features()
reader = Sens()

files = {
    'thigh': '../test_data/thigh.bin',
    'trunk': '../test_data/trunk.bin',
    'calf': '../test_data/calf.bin',
    'arm': '../test_data/arm.bin',
}

for name, file in files.items():
    df = reader.from_file(file)

    # df['acc_x'] = -df['acc_x'] # Setting incorrect sensor placement
    # df.iloc[20000:200000] = 0 # Imputting non-wear
    # df.iloc[0:250000] = 0 # Imputting non-wear

    f = features.extract(df)
    f.to_parquet(f'../temp/{name}_features.parquet')

In [ ]:
from acti_motus.classifications.references import References

references = References(
    **{
        'thigh': {
            'value': -0.201,
            'expires': '2024-09-03 12:05:51+00:00',
        },
        'trunk': {
            'value': [1.320, 0.247, -0.038],
            'expires': '2024-09-04 13:05:51+00:00',
        },
        'calibrations': [
            {
                'start': '2024-09-03 08:08:51+00:00',
                'end': '2024-09-03 08:09:11+00:00',
                'ttl': '24h',
            },
            {
                'start': '2024-09-02 10:36:31+00:00',
                'end': '2024-09-02 20:36:51+00:00',
                'ttl': '12h',
            },
            {
                'start': '2024-09-03 08:16:23+00:00',
                'end': '2024-09-03 10:17:03+00:00',
                'ttl': '12h',
            },
        ],
    }
)

In [3]:
import pandas as pd
from acti_motus import Thigh

file = pd.read_parquet('../temp/thigh_features.parquet')

references = None
activities = Thigh('Sens').detect_activities(file, references)
activities.to_parquet('../temp/thigh_activities.parquet')

In [2]:
from acti_motus import Trunk
import pandas as pd

activities = pd.read_parquet('../temp/thigh_activities.parquet')
features = pd.read_parquet('../temp/trunk_features.parquet')

references = None
df = Trunk().detect_activities(features, activities, references)
df.to_parquet('../temp/trunk_activities.parquet')

In [5]:
import logging
from acti_motus import Calf
import pandas as pd

# Configure logging to see info messages
logging.basicConfig(level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')

activities = pd.read_parquet('../temp/trunk_activities.parquet')
features = pd.read_parquet('../temp/calf_features.parquet')

df = Calf().detect_activities(features, activities)
df.to_parquet('../temp/calf_activities.parquet')
df['activity'].value_counts()

acti_motus.classifications.sensor - INFO - Non-wear detection: 0 days 00:00:00 (0.00%) out of 2 days 22:07:52 classified as non-wear time.
acti_motus.classifications.sensor - INFO - No sensor flip detected. No changes made to the data.


activity
sit         89445
lie         88311
stand       33349
move        18146
walk        11480
run          4558
bicycle      4099
stairs       1602
kneel         817
squat         666
row             0
non-wear        0
Name: count, dtype: int64

In [6]:
from acti_motus import Arm
import pandas as pd

features = pd.read_parquet('../temp/arm_features.parquet')
df = Arm().detect_activities(features)

df.value_counts(dropna=False)

acti_motus.classifications.sensor - INFO - Non-wear detection: 0 days 07:15:06 (10.34%) out of 2 days 22:08:25 classified as non-wear time.
acti_motus.classifications.sensor - INFO - No sensor flip detected. No changes made to the data.
acti_motus.classifications.sensor - INFO - No sensor flip detected. No changes made to the data.


arm_inclination
NaN                26107
78.099113           4581
79.898682           3711
79.497322           2497
91.786804           1208
                   ...  
17.182695              1
17.182545              1
17.180557              1
17.179968              1
17.190186              1
Name: count, Length: 130585, dtype: int64